In [1]:
"""
Модуль для системы вопросно-ответных задач (QA), включая:
- Загрузку и предобработку данных
- Дообучение SentenceBERT для оценки схожести текстов
- Извлечение релевантного контекста
- Выбор наилучшего ответа на вопрос
- Оценку качества модели
"""

import os
import re
from typing import Tuple, List, Dict, Any
import torch
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import f1_score
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from torch.utils.data import Dataset

In [2]:
class DatasetLoader:
    """
    Класс для загрузки и проверки данных из JSONL файлов.
    
    Attributes:
        data_dir (str): Путь к директории с данными
        val_path (str): Полный путь к файлу валидационных данных
        test_path (str): Полный путь к файлу тестовых данных
    """
    def __init__(self, data_dir: str = "data"):
        """
        Инициализирует загрузчик данных для валидационного и тестового наборов.
        
        Args:
            data_dir (str): Путь к директории, содержащей val.jsonl и test.jsonl
        """
        self.data_dir = data_dir
        self.val_path = os.path.join(data_dir, "val.jsonl")
        self.test_path = os.path.join(data_dir, "test.jsonl")
        
        # Проверка существования файлов
        if not os.path.exists(self.val_path):
            raise FileNotFoundError(f"Файл val.jsonl не найден в {data_dir}")
        if not os.path.exists(self.test_path):
            raise FileNotFoundError(f"Файл test.jsonl не найден в {data_dir}")

    def load_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Загружает val.jsonl и test.jsonl как DataFrames.
        
        Returns:
            Tuple[pd.DataFrame, pd.DataFrame]: (valid_df, test_df)
        """
        valid_df = pd.read_json(self.val_path, lines=True)
        test_df = pd.read_json(self.test_path, lines=True)
        
        print(f"Загружено:")
        print(f"- Валидационные данные: {len(valid_df)} записей")
        print(f"- Тестовые данные: {len(test_df)} записей")
        
        return valid_df, test_df

In [3]:
class TextPreprocessor:
    """Класс для предварительной обработки текста."""
    
    @staticmethod
    def split_numbered_sentences(text: str) -> List[str]:
        """
        Разделяет текст на предложения, которые начинаются с цифр в скобках (например, "(1)").
        
        Args:
            text (str): Входной текст для разделения
            
        Returns:
            List[str]: Список предложений без номеров
        """
        if not isinstance(text, str):
            return []
        text = ' '.join(text.split())  # Удаление лишних пробелов
        sentences = re.split(r'(?<!\()\s*\(\d+\)\s*', text)
        return [s.strip() for s in sentences if s.strip()]

In [4]:
class QADataset(Dataset):
    """
    Кастомный Dataset класс для хранения токенизированных данных и меток.
    Наследуется от torch.utils.data.Dataset.
    """
    def __init__(self, encodings: Dict[str, List[int]], labels: List[int]):
        """
        Args:
            encodings (Dict): Токенизированные тексты (input_ids, attention_mask и т.д.)
            labels (List): Список меток для обучения
        """
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        """Возвращает элемент датасета по индексу."""
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self) -> int:
        """Возвращает количество элементов в датасете."""
        return len(self.labels)

In [5]:
class SentenceBERT:
    """
    Класс для работы с SentenceBERT моделью:
    - Генерация эмбеддингов предложений
    - Расчет схожести между предложениями
    - Дообучение на специфичных данных
    """
    def __init__(self,
                 model_name: str = "DeepPavlov/rubert-base-cased",
                 train_data: List[Tuple[str, str, int]] = None):
        """
        Args:
            model_name (str): Название предобученной модели
            train_data (List): Данные для дообучения в формате [(текст1, текст2, метка), ...]
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        
        if train_data:
            self._train_model(model_name, train_data)
    
    def _train_model(self, model_name: str, train_data: List[Tuple[str, str, int]]):
        """Дообучение модели на предоставленных данных."""
        print("Дообучение SentenceBERT...")
        
        # Подготовка данных
        texts = [f"{x[0]} [SEP] {x[1]}" for x in train_data]
        labels = [x[2] for x in train_data]
        
        # Токенизация
        encodings = self.tokenizer(texts, truncation=True, padding=True, max_length=128)
        dataset = QADataset(encodings, labels)
        
        # Модель для классификации
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,  
            num_labels=2
        ).to(self.device)
        
        # Параметры обучения
        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=3,
            per_device_train_batch_size=60,
            save_steps=10_000,
            save_total_limit=2,
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset,
        )
        
        trainer.train()
        self.model = model.base_model  # Используем базовую модель после обучения
    
    def embed(self, sentences: List[str]) -> torch.Tensor:
        """
        Генерирует эмбеддинги для списка предложений.
        
        Args:
            sentences (List[str]): Список предложений для эмбеддинга
            
        Returns:
            torch.Tensor: Тензор с эмбеддингами предложений
        """
        inputs = self.tokenizer(
            sentences, 
            padding=True, 
            truncation=True, 
            max_length=512, 
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        return self._mean_pooling(outputs, inputs['attention_mask'])
    
    @staticmethod
    def _mean_pooling(model_output: Any, attention_mask: torch.Tensor) -> torch.Tensor:
        """
        Применяет mean pooling к выходу модели для получения эмбеддингов предложений.
        
        Args:
            model_output: Выход модели BERT
            attention_mask: Маска внимания
            
        Returns:
            torch.Tensor: Усредненные эмбеддинги
        """
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = (
            attention_mask
            .unsqueeze(-1)
            .expand(token_embeddings.size())
            .float()
        )
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    def similarity(self, sentence1: str, sentence2: str) -> float:
        """
        Вычисляет косинусную схожесть между двумя предложениями.
        
        Args:
            sentence1 (str): Первое предложение
            sentence2 (str): Второе предложение
            
        Returns:
            float: Косинусная схожесть между -1 и 1
        """
        emb1, emb2 = self.embed([sentence1, sentence2])
        return torch.cosine_similarity(emb1, emb2, dim=0).item()

In [6]:
class QASystem:
    """
    Основной класс вопросно-ответной системы.
    Объединяет все компоненты для обработки вопросов и выбора ответов.
    """
    def __init__(self, need_tune: bool = True, data_path: str = "data"):
        """
        Args:
            need_tune (bool): Нужно ли дообучать SentenceBERT
            data_path (str): Путь к директории с данными
        """
        self.dataset_loader = DatasetLoader(data_path)
        self.text_preprocessor = TextPreprocessor()
        
        # Подготовка данных и инициализация
        train_data = self._prepare_training_data()
        self.sbert = SentenceBERT(train_data=train_data if need_tune else None)
        
        # Инициализация QA модели
        self.qa_model = pipeline(
            "question-answering",
            model="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru",
            device=0 if torch.cuda.is_available() else -1,
            max_answer_len=500,
            handle_impossible_answer=False
        )
    
    def _prepare_training_data(self) -> List[Tuple[str, str, int]]:
        """
        Подготавливает данные для дообучения SentenceBERT.
        
        Returns:
            List[Tuple[str, str, int]]: Список пар текстов с метками
        """
        train_df, _ = self.dataset_loader.load_data()
        training_pairs = []
        
        for _, row in train_df.iterrows():
            passage = row['passage']
            text = passage['text']
            questions = passage['questions']
            
            for q in questions:
                question = q['question']
                answers = q['answers']
                
                pos_answers = [a['text'] for a in answers if a['label'] == 1]
                neg_answers = [a['text'] for a in answers if a['label'] == 0]
                
                # Пары вопрос-ответ
                for pos in pos_answers:
                    training_pairs.append((question, pos, 1))
                for neg in neg_answers:
                    training_pairs.append((question, neg, 0))
                
                # Пары ответ-контекст
                sentences = self.text_preprocessor.split_numbered_sentences(text)
                if sentences:
                    context = ' '.join(sentences[:3])
                    for pos in pos_answers:
                        training_pairs.append((pos, context, 1))
                    for neg in neg_answers:
                        training_pairs.append((neg, context, 0))
        
        return training_pairs
    
    def get_relevant_context(self, question: str, full_text: str, top_n: int = 7) -> str:
        """
        Извлекает наиболее релевантные предложения из текста для заданного вопроса.
        
        Args:
            question (str): Вопрос
            full_text (str): Полный текст для поиска контекста
            top_n (int): Количество возвращаемых предложений
            
        Returns:
            str: Конкатенированные наиболее релевантные предложения
        """
        sentences = self.text_preprocessor.split_numbered_sentences(full_text)
        if not sentences:
            return ""
        
        # Вычисляем схожесть каждого предложения с вопросом
        similarities = []
        for sent in sentences:
            try:
                sim = self.sbert.similarity(question, sent)
                similarities.append(sim)
            except:
                similarities.append(0)
        
        # Выбираем топ-N наиболее релевантных предложений
        top_indices = np.argsort(similarities)[-top_n:][::-1]
        return ' '.join([sentences[i] for i in top_indices])
    
    def select_best_answer(self, question: str, context: str, answer_options: List[Dict]) -> Tuple[Dict, float]:
        """
        Выбирает наилучший ответ из предложенных вариантов.
        
        Args:
            question (str): Вопрос
            context (str): Контекст для поиска ответа
            answer_options (List[Dict]): Список вариантов ответов
            
        Returns:
            Tuple[Dict, float]: Лучший ответ и его схожесть с предсказанием модели
        """
        try:
            qa_result = self.qa_model(question=question, context=context)
            model_answer = qa_result["answer"]
        except:
            model_answer = ""
        
        # Сравниваем с вариантами ответов
        best_answer = max(
            answer_options,
            key=lambda x: self.sbert.similarity(model_answer, x["text"])
        )
        similarity = self.sbert.similarity(model_answer, best_answer["text"])
        
        return best_answer, similarity
    
    def process_passage(self, passage: Dict) -> List[Dict]:
        """
        Обрабатывает один отрывок текста со всеми вопросами.
        
        Args:
            passage (Dict): Отрывок текста с вопросами
            
        Returns:
            List[Dict]: Результаты обработки всех вопросов
        """
        results = []
        text = passage["text"]
        
        for question_data in passage["questions"]:
            question = question_data["question"]
            answer_options = question_data["answers"]
            
            context = self.get_relevant_context(question, text)
            best_answer, similarity = self.select_best_answer(question, context, answer_options)
            
            correct_answers = [
                {"text": ans["text"], "idx": ans["idx"]} 
                for ans in answer_options if ans["label"] == 1
            ]
            
            results.append({
                'question': question,
                'context': context,
                'selected_answer': best_answer["text"],
                'selected_answer_idx': best_answer["idx"],
                'is_correct': bool(best_answer["label"]),
                'confidence': similarity,
                'correct_answers': correct_answers,
                'correct_answers_text': [ans["text"] for ans in correct_answers],
                'correct_answers_idx': [ans["idx"] for ans in correct_answers]
            })
        
        return results
    
    def calculate_metrics(self, y_true: List[int], y_pred: List[int]) -> Dict[str, float]:
        """
        Вычисляет метрики качества модели.
        
        Args:
            y_true (List): Истинные метки
            y_pred (List): Предсказанные метки
            
        Returns:
            Dict: Словарь с метриками (f1, accuracy)
        """
        return {
            'f1': f1_score(y_true, y_pred, average='macro'),
            'accuracy': sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
        }
    
    def valid(self, dataset: List[Dict]) -> Tuple[pd.DataFrame, Dict]:
        """
        Оценивает качество модели на валидационном наборе.
        
        Args:
            dataset (List): Список отрывков с вопросами
            
        Returns:
            Tuple: DataFrame с результатами и словарь метрик
        """
        results = []
        y_true = []
        y_pred = []
        
        for row in tqdm(dataset, desc="Оценка"):
            passage_results = self.process_passage(row['passage'])
            results.extend(passage_results)
            
            for res in passage_results:
                y_true.append(res['is_correct'])
                y_pred.append(1 if res['confidence'] > 0.5 else 0)
        
        metrics = self.calculate_metrics(y_true, y_pred)
        print(f"\nМетрики качества:")
        print(f"F1-мера: {metrics['f1']:.4f}")
        print(f"Точность: {metrics['accuracy']:.4f}")
        print(f"Проанализировано {len(results)} вопросов")
        
        return pd.DataFrame(results), metrics
    
    def eval(self, dataset: List[Dict], output_file: str = "predictions.jsonl") -> str:
        """
        Генерирует предсказания для тестового набора и сохраняет в файл.
        
        Args:
            dataset (List): Список отрывков с вопросами
            output_file (str): Путь для сохранения результатов
            
        Returns:
            str: Сообщение о завершении
        """
        results = []
        
        for row in tqdm(dataset, desc="Генерация предсказаний"):
            passage = row['passage']
            text = passage['text']
            
            for question_data in passage["questions"]:
                question = question_data["question"]
                answer_options = question_data["answers"]
                
                context = self.get_relevant_context(question, text)
                best_answer, similarity = self.select_best_answer(question, context, answer_options)
                
                results.append({
                    'question_idx': question_data['idx'],
                    'question': question,
                    'context': context,
                    'selected_answer': best_answer["text"],
                    'selected_answer_idx': best_answer["idx"],
                    'confidence': similarity,
                    'all_answers': [
                        {'text': ans['text'], 'idx': ans['idx']} 
                        for ans in answer_options
                    ]
                })
        print(results)
        
        # Сохраняем в JSONL файл
        with open(output_file, 'w', encoding='utf-8') as f:
            for res in results:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')
        
        return f"Предсказания сохранены в {output_file}"

In [7]:
def main():
    """Основная функция для запуска QA системы."""
    qa_system = QASystem(need_tune=False)
    valid_df, test_df = qa_system.dataset_loader.load_data()
    
    # Оценка на тестовых данных (с проверкой правильности)
    # P.S. эксперименты проводились на этапе подбора моделей и гиперпараметров
    valid_results, metrics = qa_system.valid(valid_df.to_dict('records'))
    valid_results.to_csv("qa_results.csv", index=False)
    
    # Сохранение предсказаний без проверки правильности
    qa_system.eval(test_df.to_dict('records'), "predictions.jsonl")

In [8]:
if __name__ == "__main__":
    main()

Загружено:
- Валидационные данные: 100 записей
- Тестовые данные: 322 записей


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Загружено:
- Валидационные данные: 100 записей
- Тестовые данные: 322 записей


Оценка: 100%|█████████████████████████████████████████████| 100/100 [09:49<00:00,  5.89s/it]



Метрики качества:
F1-мера: 0.6524
Точность: 0.7164
Проанализировано 529 вопросов


Генерация предсказаний: 100%|█████████████████████████████| 322/322 [32:27<00:00,  6.05s/it]
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

